## Ollama Function calling

In [14]:
from langchain_core.tools import tool
from langchain_ollama.chat_models import ChatOllama
from datetime import datetime
from langchain_core.output_parsers import JsonOutputParser

In [79]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def subtract(a: int, b: int) -> int:
    """Subtracts a and b."""
    return a - b

@tool
def devide(a: float, b: float) -> float:
    """Devides a and b."""
    return a / b

@tool
def use_Internet(query:str)->str:
    """ Use internet if there is a shortage of information or anything related to upto date information"""
    return query

@tool
def date(a:str)->str:
    """ Tells todays date and time """
    return datetime.now()

tools = [add, subtract, multiply, devide, use_Internet, date]

In [33]:
from langchain.tools.render import render_text_description
render = render_text_description([add, subtract, multiply, devide, use_Internet, date])

In [43]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([('system',f"you are an ai assistant ,here is the available tools with description {render}, for the give input retrun the name and input of the tool to use.Return your json object blob with both 'name' and 'arguments'keys.and also make sure that returnd argument is in dictonery formate.do not return anything else") ,('user',"{input}")])

In [44]:
prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="you are an ai assistant ,here is the available tools with description add(a: int, b: int) -> int - Adds a and b.\nsubtract(a: int, b: int) -> int - Subtracts a and b.\nmultiply(a: int, b: int) -> int - Multiplies a and b.\ndevide(a: float, b: float) -> float - Devides a and b.\nuse_Internet(a: str) -> str - Use internet if there is a shortage of information or anything related to upto date information\ndate(a: str) -> str - Tells todays date and time, for the give input retrun the name and input of the tool to use.Return your json object blob with both 'name' and 'arguments'keys.and also make sure that returnd argument is in dictonery formate.do not return anything else")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [73]:
model =ChatOllama(model="mistral:instruct",keep_alive=1,format='json')
llm_with_tools = model.bind_tools(tools)
def selector(response):
    return globals()[response['name']](response['arguments'])

In [56]:
query = "What is 3 * 12 + 11 - 49?"


In [77]:
chain1 = prompt | model | JsonOutputParser()

In [74]:
chain = prompt | model | JsonOutputParser() | selector

In [80]:
query = "What is current stock price of tata steel?"
print(chain1.invoke({"input":query}))

{'name': 'use_Internet', 'arguments': {'query': 'current stock price of tata steel'}}


In [81]:
query = "What is 1+2"
print(chain1.invoke(query))

{'name': 'add', 'arguments': {'a': 1, 'b': 2}}


In [82]:
query = "What is current stock price of tata steel?"
print(chain.invoke({"input":query}))

current stock price of tata steel


In [84]:
query = "What is sum of 199999999 and 280000000"
print(chain.invoke(query))

479999999


<div style="text-align: center;">
    <p style="font-size: 1em; margin: 0;">Written by Ramachandra Udupa | <a href="https://www.linkedin.com/in/ramachandra-udupa/" style="color: #4CAF50; text-decoration: none;">Connect on LinkedIn</a></p>
</div>